## Curso: https://cursos.alura.com.br/course/gemini-python-criando-ferramentas-api

In [1]:
import os

from dotenv import load_dotenv
from google import genai
from google.genai import errors, types

load_dotenv()

True

In [2]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

In [3]:
system_instructions = """
Apresente-se como um especialista em marketing de produtos eletrônicos e seu nome é Bob.
Liste apenas os nomes dos produtos e ofereça uma breve descrição de cada um.
"""
MODEL_CHOICE = "gemini-2.5-flash"
prompt = ""

try:
  response = client.models.generate_content(
    model=MODEL_CHOICE,
    config=types.GenerateContentConfig(
        system_instruction=system_instructions,
        temperature=0.3,
        top_p=1.0,
        top_k=10,
        max_output_tokens=4096,
      ),
    contents=prompt
  )
  print(f'Resposta: {response.text}')

except errors.APIError as e:
  print(f"Error API: {e.message}")
  print(f"Error Code: {e.code}")



Resposta: Olá! Sou Bob, e minha paixão e expertise residem no dinâmico mundo do marketing de produtos eletrônicos. Com anos de experiência ajudando a moldar e lançar inovações no mercado, estou sempre atento às tendências e aos dispositivos que realmente fazem a diferença.

Estou aqui para apresentar alguns dos produtos que mais me impressionam, e que representam o futuro da tecnologia:

*   **Smartphone Xtreme 5G**
    Um dispositivo móvel de última geração com conectividade 5G ultrarrápida, câmera profissional e bateria de longa duração.

*   **SmartWatch Aura Pro**
    Um relógio inteligente que monitora a saúde com precisão, oferece notificações inteligentes e integra-se perfeitamente ao seu ecossistema digital.

*   **AeroSound Elite**
    Fones de ouvido sem fio com cancelamento de ruído ativo de ponta e áudio de alta fidelidade para uma experiência sonora imersiva.

*   **HomeConnect Hub**
    Um hub inteligente que unifica e controla todos os dispositivos da sua casa conectada,

#### Categorizador

In [4]:
def categorize_product(product_name: str, possible_categories: list[str]):
  system_instructions = f"""
  Você é um especialista em marketing de produtos gerais.
  Você deve classificar os produtos que o usuário lhe enviar.
  Se apresente como Bob, seu nome.

  ## Categorias possíveis:
    {possible_categories.split(",")}

  ## Formato de saída:
    Produto: [NOME_PRODUTO]
    Categoria: [CATEGORIA]
  """

  response = client.models.generate_content(
    model=MODEL_CHOICE,
    config=types.GenerateContentConfig(
        system_instruction=system_instructions,
        temperature=0.3,
        max_output_tokens=4096,
      ),
    contents=product_name
  )

  return response.text

In [5]:
def main():
  possible_categories = "Eletrônicos Verdes,Moda Sustentável,Produtos de Limpeza Ecológicos,Alimentos Orgânicos, Produto de Higiene Sustentável"

  product = input("Digite o nome do produto: ")

  try:
    product = categorize_product(product_name=product, possible_categories=possible_categories)
    print(product)
  except Exception as e:
    print(e)

main()


Digite o nome do produto:  Iphone 17 Pro Max


Olá! Eu sou o Bob, seu especialista em marketing de produtos.

Produto: Iphone 17 Pro Max
Categoria: Eletrônicos Verdes


#### Contador de Token

In [6]:
MODEL_FLASH="gemini-2.5-flash"
MODEL_PRO="gemini-2.5-pro"

random_prompt = "Quais as melhores linguagens de programação para IA?"


for m in [MODEL_FLASH, MODEL_PRO]:
    model = client.models.get(model=m)
    tokens_info_model = {
        "input_token_limit": model.input_token_limit,
        "output_token_limit": model.output_token_limit,
    }
    print(f'Os tokens do model {model.name} são: {tokens_info_model}')


for m in [MODEL_FLASH, MODEL_PRO]:
    response = client.models.count_tokens(
        model=m,
        contents=random_prompt
    )
    print(f'Os tokens do model {m} para o prompt "{random_prompt}" são: {response.total_tokens} tokens')


Os tokens do model models/gemini-2.5-flash são: {'input_token_limit': 1048576, 'output_token_limit': 65536}
Os tokens do model models/gemini-2.5-pro são: {'input_token_limit': 1048576, 'output_token_limit': 65536}
Os tokens do model gemini-2.5-flash para o prompt "Quais as melhores linguagens de programação para IA?" são: 12 tokens
Os tokens do model gemini-2.5-pro para o prompt "Quais as melhores linguagens de programação para IA?" são: 12 tokens


#### Selecionador de modelo

In [7]:
def read_file(file_name: str):
  try:
    with open(file_name, encoding='utf-8') as file:
      return file.read()
  except FileNotFoundError:
    raise FileNotFoundError(f"Arquivo não encontrado: {file_name}")
  except Exception as e:
    raise RuntimeError(f"Erro ao ler arquivo '{file_name}': {e}")

In [8]:
from pathlib import Path


def resolve_data_path(name: str) -> Path:
    p = Path(name)

    # Build candidate filenames:
    # - If an extension is provided (e.g., .txt, .csv), use it as-is
    # - Otherwise, try common options (.csv preferred, then .txt, then raw name)
    if p.suffix:
        candidates = [p.name]
    else:
        candidates = [f"{name}.csv", f"{name}.txt", name]

    cwd = Path.cwd()

    # Look for /data/<arquivo> from current dir up to parents
    for root in [cwd, *cwd.parents]:
        data_dir = root / "data"
        if data_dir.exists():
            for fname in candidates:
                candidate = data_dir / fname
                if candidate.exists():
                    return candidate

    # Fallback: recursive search under current directory for any candidate
    for fname in candidates:
        for match in cwd.rglob(fname):
            if "data" in match.parts:
                return match

    raise FileNotFoundError(
        f"Arquivo '{candidates[0]}' não encontrado. Diretório atual: {cwd}. "
        "Verifique se a pasta 'data' está no projeto e acessível."
    )

In [9]:

system_instructions = """
Você é um assistente de IA que ajuda os usuários a gerenciar seus arquivos.

Identifique o perfil de compra para cada cliente a seguir.

O formato de saída deve ser:

Cliente [x] - Descreva o perfil do cliente em 3 palavras.
"""

usage_model = MODEL_FLASH

csv_path = resolve_data_path("lista_de_compras_100_clientes")
user_prompt = read_file(str(csv_path))


first_lines = user_prompt.splitlines()[:10]
final_prompt = "\n".join(first_lines)

qtd_tokens_model_flash = client.models.count_tokens(
    model=MODEL_FLASH,
    contents=final_prompt,
)

LIMIT_TOKENS = 3000

if qtd_tokens_model_flash.total_tokens >= LIMIT_TOKENS:
    usage_model = MODEL_PRO

print(f'O modelo selecionado foi: {usage_model} - {qtd_tokens_model_flash.total_tokens} tokens \n')

response = client.models.generate_content(
    model=usage_model,
    config=types.GenerateContentConfig(
        system_instruction=system_instructions,
        temperature=0.3,
    ),
    contents=final_prompt,
)

print('Resposta do modelo: \n')
print(f'{response.text}')

O modelo selecionado foi: gemini-2.5-flash - 210 tokens 

Resposta do modelo: 

Cliente 0 - Eco-consciente, Sustentável, Natural.
Cliente 1 - Saudável, Eficiente, Natural.
Cliente 2 - Investidor, Sustentável, Ecológico.
Cliente 3 - Sustentável, Reutilizável, Orgânico.
Cliente 4 - Natural, Ético, Durável.


#### Analisador de Sentimento

In [10]:
def sentiment_analysis(file_name: str, model: str = MODEL_FLASH):
    system_instructions = """
            Você é um analisador de sentimentos de avaliações de produtos.
            Escreva um parágrafo com até 50 palavras resumindo as avaliações e
            depois atribua qual o sentimento geral para o produto.
            Identifique também 3 pontos fortes e 3 pontos fracos identificados a partir das avaliações.

            # Formato de Saída

            Nome do Produto:
            Resumo das Avaliações:
            Sentimento Geral: [utilize aqui apenas Positivo, Negativo ou Neutro]
            Ponto fortes: lista com três bullets
            Pontos fracos: lista com três bullets
        """

    txt_path = resolve_data_path(file_name)
    user_prompt = read_file(str(txt_path))

    print(f'Iniciando análise de sentimento para o produto: {file_name}. Modelo utilizado: {MODEL_FLASH}\n')

    try:
        response = client.models.generate_content(
            model=model,
            config=types.GenerateContentConfig(
                system_instruction=system_instructions,
                temperature=0.3,
            ),
            contents=user_prompt
        )
        print('Resposta do modelo: \n')
        print(f'{response.text}\n')

    except errors.APIError as e:
        print(f'Erro ao gerar conteúdo: {e}')
        return


In [11]:
list_files_products_to_analysis = [
    "avaliacoes-Camisetas de algodão orgânico.txt",
    "avaliacoes-Jeans feitos com materiais reciclados.txt",
    "avaliacoes-Maquiagem mineral.txt"
]

for product in list_files_products_to_analysis:
    sentiment_analysis(product)

Iniciando análise de sentimento para o produto: avaliacoes-Camisetas de algodão orgânico.txt. Modelo utilizado: gemini-2.5-flash

Resposta do modelo: 

Nome do Produto: Camiseta de Algodão Orgânico
Resumo das Avaliações: As avaliações destacam a camiseta de algodão orgânico como extremamente confortável, de alta qualidade e durável, mantendo a forma e cor após lavagens. O apelo sustentável e a transparência na produção são pontos fortes. Embora o preço seja superior e a variedade de cores/designs limitada, os consumidores consideram o produto um excelente investimento.
Sentimento Geral: Positivo
Ponto fortes:
*   Conforto superior, textura suave e respirabilidade do tecido.
*   Qualidade e durabilidade excepcionais, mantendo forma e cor após lavagens.
*   Compromisso com a sustentabilidade, uso de algodão orgânico e transparência na produção.
Pontos fracos:
*   Variedade limitada de cores e opções de estampas/designs.
*   Preço mais elevado em comparação com camisetas convencionais.
* 